<a href="https://colab.research.google.com/github/zzyy-gh/stonks-lambda-/blob/main/delete_meta_by_dates.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
def lambda_handler(event, context):
    table_name = 'ticker_meta'
    results = []
    last_evaluated_key = None
    
    # find uuid
    while True:
        try:
            if last_evaluated_key:
                response = client.scan(
                    TableName=table_name,
                    ExclusiveStartKey=last_evaluated_key,
                    ExpressionAttributeNames={
                        '#t': 'tkr',
                        '#u': 'uuid',
                        '#d': 'date',
                    },
                    ExpressionAttributeValues={
                        ':dd': {
                            'N': '20210915',
                        },
                    },
                    FilterExpression='#d < :dd',
                    ProjectionExpression='#t, #u, #d',
                )
            else: 
                response = client.scan(
                    TableName=table_name,
                    ExpressionAttributeNames={
                        '#t': 'tkr',
                        '#u': 'uuid',
                        '#d': 'date',
                    },
                    ExpressionAttributeValues={
                        ':dd': {
                            'N': '20210915',
                        },
                    },
                    FilterExpression='#d < :dd',
                    ProjectionExpression='#t, #u, #d',
                    )
            last_evaluated_key = response.get('LastEvaluatedKey')
            
            results.extend(response['Items'])
            
            if not last_evaluated_key:
                break
        except:
            print('aww shit')
    
    cleaned = []
    for data in results:
        cleaned.append(
            {
                "DeleteRequest": {
                    "Key": {
                        "uuid": data["uuid"]
                    }
                }
            }
            ) 
    
    # batch delete
    def delete(results):
        strCleanList = [ [] for _ in range((len(results) - 1) // 25 + 1) ]
        unprocList = []
        for i, data in enumerate(results):
            strCleanList[i // 25].append(data)
        for batch in strCleanList:
            try: 
                response = client.batch_write_item(
                    RequestItems= {
                        'ticker_meta': batch
                    }
                )
                if bool(response['UnprocessedItems']):
                    print(response)
                    unprocList.extend(response['UnprocessedItems']['ticker_meta'])
            except Exception as e:
                print(batch[0])
                print(e)
        if len(unprocList) > 0:
            print(str(len(unprocList)) + ' unprocessed items.')
            delete(unprocList)
    
    print('Start deletion.')
    delete(cleaned)
    return